***
***
<span style="font-size: 28pt"> <span style="color:blue">
    **Step.3.3_Dynamic Modeling on Cells**
    </span><br>
 
***


# set up the packages

In [6]:
import scvelo as scv
scv.logging.print_version()

scv.settings.verbosity = 3  # show errors(0), warnings(1), info(2), hints(3)
scv.settings.presenter_view = True  # set max width size for presenter view
scv.set_figure_params('scvelo')  # for beautified visualization


Running scvelo 0.2.2 (python 3.6.10) on 2021-03-14 21:37.


# read data from previously saved merged loom file

This taks from 9:50 to 

In [7]:
import pickle
with open('../ProcessedData/Steady_merged_cells.pickle',"rb") as f:
    adata = pickle.load(f)

In [10]:
# show proportions of spliced/unspliced abundances
scv.utils.show_proportions(adata)

Abundance of ['spliced', 'unspliced']: [0.8 0.2]


If you have a very large datasets, you can save memory by clearing attributes not required via `scv.utils.cleanup(adata)`.

## Preprocess the data

Preprocessing that is necessary consists of :
- gene selection by **detection** (detected with a minimum number of counts) and **high variability** (dispersion).
- **normalizing** every cell by its initial size and **logarithmizing** X.

Filtering and normalization is applied in the same vein to spliced/unspliced counts and X.
Logarithmizing is only applied to X. <br>If X is already preprocessed from former analysis, it won't touch it.

All of this is summarized in a single function `pp.filter_and_normalize`, which basically runs the following:

```
scv.pp.filter_genes(adata, min_shared_counts=10)
scv.pp.normalize_per_cell(adata)
scv.pp.filter_genes_dispersion(adata, n_top_genes=3000)
scv.pp.log1p(adata)
```


this step take from 2:07--2:12pm, 5 minutes

## Compute velocity and velocity graph
The gene-specific velocities are obtained by fitting a ratio between precursor (unspliced) and mature (spliced) mRNA abundances that well explains the steady states (constant transcriptional state) and then computing how the observed abundances deviate from what is expected in steady state. 
## (**We will soon release a version that does not rely on the steady state assumption anymore**).

Every tool has its plotting counterpart. The results from `scv.tl.velocity`, for instance, can be visualized using `scv.pl.velocity`.

from loading to computation:
9:13--9:33am

In [ ]:
scv.tl.recover_dynamics(adata)

recovering dynamics
... 0%

In [ ]:
scv.tl.velocity(adata,mode='dynamical')

<span style="font-size: 10pt"> <span style="color:blue">
This computes the (cosine) correlation of potential cell transitions with the velocity vector in high dimensional space.<br>
The resulting velocity graph has dimension $n_{obs} \times n_{obs}$ and summarizes the possible cell state changes (given by a transition from one cell to another) that are well explained through the velocity vectors. <br>If you set `approx=True` it is computed on a reduced PCA space with 50 components. 
The velocity graph can then be converted to a transition matrix by applying a Gaussian kernel on the cosine correlation which assigns high probabilities to cell state changes that correlate with the velocity vector. You can access the Markov transition matrix via `scv.tl.transition_matrix`.<br> The resulting transition matrix can be used for a variety of applications shown hereinafter. For instance, it is used to place the velocities into a low-dimensional embedding by simply applying the mean transition with respect to the transition probabilities, i.e. `scv.tl.velocity_embedding`. <br>Further, we can trace cells back along the Markov chain to their origins and potential fates, thus obtaining root cells and end points within a trajectory; via `scv.tl.terminal_states`.    
</span><br>

2:12pm -- 2:13pm

import pickle
with open('../ProcessedData/cells_Dynamic_Modeling.pickle', 'wb') as f:
    pickle.dump(adata, f)

In [ ]:
scv.tl.velocity_graph(adata)

# Plot results

Finally, the velocities are projected onto any embedding specified in `basis` and visualized in one of three available ways: on single cell level, on grid level, or as streamplot as shown here.

 strange, even though I have the umap coordinate, there is still problem with get the X_umap.<br>
ref: https://github.com/theislab/scvelo/issues/170

## stream plots

In [ ]:
scv.pl.velocity_embedding_stream(adata, basis='umap_cell_embeddings', color=['percent_mt','RNA_snn_res_1'],legend_fontweight=1,save = 'scvelo_umap_overlaySeuratClusters.pdf')

## grid plot

In [ ]:
scv.pl.velocity_embedding_grid(adata, color='Tmsb10', basis='umap_cell_embeddings',arrow_length =5,
                               layer=['velocity', 'spliced'], arrow_size=1, figsize=(8,5),save = 'Seurat_umap_overlaySeuratClusters.pdf')

# Explore velocity genes

## rank velocity genes by clusters identified with scv.tl.umap() function

In [ ]:
scv.tl.rank_velocity_genes(adata, groupby='seurat_clusters')
scv.DataFrame(adata.uns['rank_velocity_genes']['names']).head()


In [ ]:
scv.tl.rank_velocity_genes(adata, groupby='Clusters')
scv.DataFrame(adata.uns['rank_velocity_genes']['names']).head()

In [ ]:
# var_names = ["S100a6","Anxa1","Bst2","Phf11b","Myb","Nr4a3","Cx3cr1"]
# scv.pl.velocity(adata, var_names=var_names, colorbar=True, ncols=2, basis='umap_cell_embeddings',figsize=(8,6),mode=None)

# dynamic modeling

this step takes about 5 minutes to complete

In [ ]:
scv.pl.scatter(adata, basis=['umap','umap_cell_embeddings'], color='velocity_pseudotime', color_map='gnuplot', size=80)

In [ ]:
scv.tl.latent_time(adata)

In [ ]:
scv.pl.scatter(adata, basis=['umap','umap_cell_embeddings'], color='latent_time', color_map='gnuplot', size=80)

In [ ]:
#scv.pl.scatter(adata, basis='umap_cell_embeddings', color='latent_time', color_map='gnuplot', size=80)

In [ ]:
scv.pl.velocity_embedding_stream(adata, basis='umap', color=['percent_mt','seurat_clusters'],legend_fontweight=1)

In [ ]:
scv.pl.velocity_embedding_stream(adata, basis='umap_cell_embeddings', color=['percent_mt','seurat_clusters'],legend_fontweight=1)

# save the adata as a pickle file.

In [ ]:
import pickle
with open('../ProcessedData/ata/Tcells_Dynamic_Modeling.pickle', 'wb') as f:
    pickle.dump(adata, f)